In [1]:
import os
from sentence_transformers import SentenceTransformer
from langchain.vectorstores.zilliz import Zilliz
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.milvus import Milvus
from langchain.schema.runnable import RunnablePassthrough
from pymilvus import connections

In [2]:
# Milvus Setup Arguments
COLLECTION_NAME = 'movies_db'  # Collection name
DIMENSION = 384  # Embeddings size
COUNT = 1000  # Number of vectors to insert
MILVUS_HOST = 'localhost'
MILVUS_PORT = '19530'

# Inference Arguments
BATCH_SIZE = 128

# Search Arguments
TOP_K = 3

In [3]:
from pymilvus import connections

# Connect to Milvus Database
connections.connect(host=MILVUS_HOST, port=MILVUS_PORT)

In [4]:
from pymilvus import utility

# Remove any previous collections with the same name
if utility.has_collection(COLLECTION_NAME):
    utility.drop_collection(COLLECTION_NAME)

In [5]:
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection


# Create collection which includes the id, title, and embedding.
fields = [
    FieldSchema(name='id', dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name='title', dtype=DataType.VARCHAR, max_length=200),
    FieldSchema(name='embedding', dtype=DataType.FLOAT_VECTOR, dim=DIMENSION)
]
schema = CollectionSchema(fields=fields)
collection = Collection(name=COLLECTION_NAME, schema=schema)

In [6]:
# Create an IVF_FLAT index for collection.
index_params = {
    'metric_type':'L2',
    'index_type':"IVF_FLAT",
    'params':{'nlist': 1536}
}
collection.create_index(field_name="embedding", index_params=index_params)
collection.load()

In [13]:
import os
from sentence_transformers import SentenceTransformer

transformer = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')


def embed_insert(data):
    embeds = transformer.encode(data[1]) 
    ins = [
            data,
            [x for x in embeds]
    ]
    collection.insert(ins)

import time

data_batch = [[],[]]

count = 0

for file in os.listdir('./movie_text_data/'):
    if count <= COUNT:
        with open(f'./movie_text_data/{file}', 'r') as f:
            content = f.read()
        data_batch[0].append(file)
        data_batch[1].append(content)
        if len(data_batch) % BATCH_SIZE == 0:
            embed_insert(data_batch)
            data_batch = [[],[]]
        count += 1
    else:
        break

if len(data_batch[0]) != 0:
    embed_insert(data_batch)

collection.flush()


DataNotMatchException: <DataNotMatchException: (code=1, message=The data don't match with schema fields, expect 1 list, got 2)>

In [69]:
search_terms = ['cars', 'monsters']

def embed_search(data):
    embeds = transformer.encode(data) 
    return [x for x in embeds]

search_data = embed_search(search_terms)

start = time.time()
res = collection.search(
    data=search_data,  # Embeded search value
    anns_field="embedding",  # Search across embeddings
    param={},
    limit = TOP_K,  # Limit to top_k results per search
    output_fields=['title']  # Include title field in result
)
end = time.time()

for hits_i, hits in enumerate(res):
    print('Title:', search_terms[hits_i])
    print('Search Time:', end-start)
    print('Results:')
    for hit in hits:
        print( hit.entity.get('title'), '----', hit.distance)
    print()


Title: cars
Search Time: 0.049751996994018555
Results:
Mabel at the Wheel ---- 1.171460747718811
Gentlemen of Nerve ---- 1.29246187210083
The Roaring Road ---- 1.3758083581924438

Title: monsters
Search Time: 0.049751996994018555
Results:
The Penalty ---- 1.4626948833465576
The Paleface ---- 1.4731868505477905
The Doorway to Hell ---- 1.4764574766159058

